# Social Network Analysis for ISIS

New to Network Analysis, but very happy to learn

## Before we get started

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [ ]:
df = pd.read_csv('../input/tweets.csv')

In [ ]:
print(df.shape)

In [ ]:
index = []
for i in range(len(df['tweets'])):
    if '@' in df['tweets'][i]:
        index.append(i)
        
With_mention = df.iloc[index,:].reset_index(drop=True)
With_mention['Tagged_User'] = With_mention['tweets'].apply(lambda x: re.findall(r'@([A-Za-z0-9_]+)',str(x)))
User = With_mention['username'].unique()
With_mention['Tagged_User_Co'] = With_mention['Tagged_User'].apply(lambda x: list(set(x).intersection(User)) )
With_mention['Co_length'] = With_mention['Tagged_User_Co'].apply(lambda x: len(x))
With_mention_2 = With_mention[With_mention['Co_length']>0].reset_index(drop=True)

## Isis Mentions Network

We first create a data frame which contains the information of individual tagging activities in each row.

In [ ]:
for i in range(len(With_mention['tweets'])):
    frame = With_mention.iloc[i,:]
    for j in range(len(frame['Tagged_User'])):
        tmp = pd.DataFrame({'User':[frame['username']],'Mentions':[frame['Tagged_User'][j]],'Time': [frame['time']],'User_numberstatuses':[frame['numberstatuses']],'User_followers':[frame['followers']],'Weight': [1]})
        if i==0 and j==0:
            Mention_net = tmp
        else:
            Mention_net = Mention_net.append(tmp, ignore_index=True)
    

Then, in order to be more prepared for some extra analysis, a similar data frame will be created that only contains the interactivity between users in the data frame.

In [ ]:
for i in range(len(With_mention_2['tweets'])):
    frame = With_mention_2.iloc[i,:]
    for j in range(len(frame['Tagged_User_Co'])):
        Mentioned_follower = list(df[df['username']==frame['Tagged_User_Co'][j]]['followers'])[0]
        Mehtioned_statuses = list(df[df['username']==frame['Tagged_User_Co'][j]]['numberstatuses'])[0] 
        tmp = pd.DataFrame({'Mentioned_statuses':[Mehtioned_statuses],'Mentioned_followers':[Mentioned_follower],'User':[frame['username']],'Mentions':[frame['Tagged_User_Co'][j]],'Time': [frame['time']],'User_numberstatuses':[frame['numberstatuses']],'User_followers':[frame['followers']],'Weight': [1]})
        if i==0 and j==0:
            Mention_net_2 = tmp
        else:
            Mention_net_2 = Mention_net_2.append(tmp, ignore_index=True)
    

In [ ]:
Mention_net = Mention_net[Mention_net['User']!=Mention_net['Mentions']]
Mention_net_2 = Mention_net_2[Mention_net_2['User']!=Mention_net_2['Mentions']]

In [ ]:
Mention_net = Mention_net.reset_index(drop=True)
Mention_net_2 = Mention_net_2.reset_index(drop=True)

**A glimpse to the new data frame**

In [ ]:
Mention_net.head(5)

## In and out degree of network

It is insightful to see how many times a user mentions others and how many times a user is mentioned by the others. For targeting the potential terrorist, in and out degree on social media may be a helpful indicators.

In [ ]:
In_degree = Mention_net.groupby(by=['Mentions'],as_index=False)['Weight'].sum()
Out_degree= Mention_net.groupby(by=['User'],as_index=False)['Weight'].sum()

In [ ]:
In_degree = pd.DataFrame(In_degree).sort_values(by='Weight',ascending=False).reset_index(drop=True)
Out_degree = pd.DataFrame(Out_degree).sort_values(by='Weight',ascending=False).reset_index(drop=True)

In [ ]:
print( 'Most mentioned user is '+str(In_degree['Mentions'][0])+' with ' + str(In_degree['Weight'][0])+' times mentioned by the other users.')
print( 'Most active user is '+str(Out_degree['User'][0])+' with ' + str(Out_degree['Weight'][0])+' times mentioning other users.')

Similarly, in the second data frame where all users and tagged users are in the original data set, we can also see the names of most active and mentioned user.

In [ ]:
In_degree_2 = Mention_net_2.groupby(by=['Mentions'],as_index=False)['Weight'].sum()
Out_degree_2= Mention_net_2.groupby(by=['User'],as_index=False)['Weight'].sum()

In [ ]:
In_degree_2 = pd.DataFrame(In_degree_2).sort_values(by='Weight',ascending=False).reset_index(drop=True)
Out_degree_2 = pd.DataFrame(Out_degree_2).sort_values(by='Weight',ascending=False).reset_index(drop=True)

In [ ]:
print( 'Most mentioned user is '+str(In_degree_2['Mentions'][0])+' with ' + str(In_degree['Weight'][0])+' times mentioned by the other users.')
print( 'Most active user is '+str(Out_degree_2['User'][0])+' with ' + str(Out_degree['Weight'][0])+' times mentioning other users.')

People should be careful with the one who is consistently active in ISIS social networks, **both actively and passively** involved should be paid with attention.

## Draw network graphs for the ISIS Twitter Network

**Network across the whole Tweet**

In [ ]:
network1 = Mention_net.iloc[:,[0,2,5]]
network1 = network1.groupby(by=['Mentions','User'],as_index='False')['Weight'].sum().reset_index(name='Weight')
network1= pd.DataFrame(network1).sort_values(by='Weight',ascending=False).reset_index(drop=True)
network1 = network1[network1['Weight']>20]
import networkx as nx

In [ ]:
print('The 10 most frequent tagged user pairs are ')
network1.iloc[0:9,:]

God! melvynlion always tag the other people on Tweet. Maybe he is a key to dig out the social network of ISIS pro!

In [ ]:
G = nx.Graph()
for i in range(len(network1['User'])):
    G.add_edge(network1['User'][i],network1['Mentions'][i],weight=network1['Weight'][i])


Let's  visualize the networks of part of the active users!

In [ ]:
plt.figure(1)
plt.figure(figsize=(14,14))
d=d = nx.degree(G)
nx.draw_circular(G,node_color='g', edge_color='#909090', node_size=[v*100 for v in d.values()],with_labels=True)
plt.axis('equal')

Some nodes have really intensive links from/to the other people, right?

**Network within the ISIS pro teams**

In [ ]:
network2 = Mention_net_2.iloc[:,[2,4,7]]
network2 = network2.groupby(by=['Mentions','User'],as_index='False')['Weight'].sum().reset_index(name='Weight')
network2= pd.DataFrame(network2).sort_values(by='Weight',ascending=False).reset_index(drop=True)
network2 = network2[network2['Weight']>10]
import networkx as nx

In [ ]:
print('The 10 most frequent tagged user pairs within the ISIS pro are ')
network2.iloc[0:9,:]

In [ ]:
G = nx.Graph()
for i in range(len(network2['User'])):
    G.add_edge(network2['User'][i],network2['Mentions'][i],weight=network2['Weight'][i])
plt.figure(2)
plt.figure(figsize=(14,14))
d=d = nx.degree(G)
nx.draw_circular(G,node_color='g', edge_color='#909090', node_size=[v*100 for v in d.values()],with_labels=True)
plt.axis('equal')

The second network is somehow sparse in comparison with the first network graph.

## Conclusion

Counting the number of tagging/being tagged may be helpful to identify the supporters of ISIS. If you are interested, you can also perform text mining on the Tweets. For the further exploration, one can cluster the Tweet users by the content of their Tweets. The text mining techniques may be helpful.